In [2]:
SAMPLE_SAME_SCANNER = """
--- scanner 0 ---
-1,-1,1
-2,-2,2
-3,-3,3
-2,-3,1
5,6,-4
8,0,7

--- scanner 0 ---
1,-1,1
2,-2,2
3,-3,3
2,-1,3
-5,4,-6
-8,-7,0

--- scanner 0 ---
-1,-1,-1
-2,-2,-2
-3,-3,-3
-1,-3,-2
4,6,5
-7,0,8

--- scanner 0 ---
1,1,-1
2,2,-2
3,3,-3
1,3,-2
-4,-6,5
7,0,8

--- scanner 0 ---
1,1,1
2,2,2
3,3,3
3,1,2
-6,-4,-5
0,7,-8
"""

SAMPLE_DIFFERENT_SCANNERS = """
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
"""

In [3]:
def tokenize_line(line):
    if line.startswith("---"):
        return "NEW_SCANNER"
    else:
        return [int(i) for i in line.split(',')]

def parse_text(raw_text):
    return [tokenize_line(l) for l in raw_text.split("\n") if l]

def read_input():
    with open("input.txt", "rt") as f:
        return f.read()

In [4]:
from dataclasses import dataclass
from collections import defaultdict
from math import sqrt, pow

In [32]:
# Solution inspired by https://github.com/Fadi88/AoC/blob/master/2021/day19/code.py

@dataclass(frozen=True)
class Beacon:
    x: int
    y: int
    z: int

    def distance_to(self, other: "Beacon"):
        xx = pow(self.x - other.x, 2)
        yy = pow(self.y - other.y, 2)
        zz = pow(self.z - other.z, 2)
        return int(sqrt(xx + yy + zz))

    def rotate(self, matrix):
        points = [self.x, self.y, self.z]
        new_points = [
            points[matrix['x'][0]] * matrix['x'][1],
            points[matrix['y'][0]] * matrix['y'][1],
            points[matrix['z'][0]] * matrix['z'][1],
        ]
        return Beacon(*new_points)

    def translate(self, translation):
        return Beacon(self.x - translation[0], self.y - translation[1], self.z - translation[2])

class Scanner:
    def __init__(self):
        self.beacons = []

    def add_beacon(self, beacon: Beacon):
        self.beacons.append(beacon)

    def add_scanner(self, other: "Scanner"):
        for b in other.beacons:
            if b not in self.beacons:
                self.beacons.append(b)

    def __repr__(self):
        return f"Scanner(beacons={self.beacons})"

    def fingerprint(self):
        result = defaultdict(set)
        for b1 in self.beacons:
            for b2 in self.beacons:
                if b1 == b2:
                    continue
                result[b1].add(b1.distance_to(b2))
        return result
    
    def transform(self, rotation_matrix, translation):
        result = Scanner()
        for b in self.beacons:
            result.add_beacon(b.rotate(rotation_matrix).translate(translation))
        return result


def how_similar(fingerprint_1, fingerprint_2):
    overlaps = []
    for v1 in fingerprint_1.values():
        for v2 in fingerprint_2.values():
            overlaps.append(len(v1 & v2))
    return max(overlaps)

def calculate_rotation_and_translation(base: Scanner, to_align: Scanner):
    fingerprint1 = base.fingerprint()
    fingerprint2 = to_align.fingerprint()

    mapping = {}
    for b1, v1 in fingerprint1.items():
        for b2, v2 in fingerprint2.items():
            if len(v1 & v2) > 10:
                mapping[b1] = b2

    # Need at least two points to find a "centre of gravity"
    cog1 = [
        sum(b.x for b in mapping.keys()) / len(mapping.keys()),
        sum(b.y for b in mapping.keys()) / len(mapping.keys()),
        sum(b.z for b in mapping.keys()) / len(mapping.keys())
    ]

    cog2 = [
        sum(b.x for b in mapping.values()) / len(mapping.keys()),
        sum(b.y for b in mapping.values()) / len(mapping.keys()),
        sum(b.z for b in mapping.values()) / len(mapping.keys())
    ]

    # p1 - first beacon from base; p2 - first beacon from to_align
    # They should represent the same point
    p1 = list(mapping.keys())[0]
    p2 = mapping[p1]

    # Remove centre of gravity to align the two points
    p1_centered = round(p1.x - cog1[0]), round(p1.y - cog1[1]), round(p1.z - cog1[2])
    p2_centered = round(p2.x - cog2[0]), round(p2.y - cog2[1]), round(p2.z - cog2[2])

    # Calculate a rotation matrix.
    # Given values for p1_centered and p2_centered of (-169, 386, -730) and (386, 730, 169)
    # This returns {'x': (2, -1), 'y': (0, 1), 'z': (1, -1)}
    rotation_matrix = {}
    for i, direction in enumerate(list('xyz')):
        p1_val = abs(p1_centered[i])
        p2_index = [abs(p) for p in p2_centered].index(p1_val)
        magnitude = p1_centered[i] // p2_centered[p2_index]
        rotation_matrix[direction] = p2_index, magnitude

    # Rotate the original p2 using the rotation matrix.
    # p2 is now equal to p1 less any residual translation
    p2_rotated = p2.rotate(rotation_matrix)

    translation = [
        p2_rotated.x - p1.x,
        p2_rotated.y - p1.y,
        p2_rotated.z - p1.z
    ]

    return rotation_matrix, translation

def make_scanners(lines):
    scanners = []
    for l in lines:
        if l == "NEW_SCANNER":
            scanners.append(Scanner())
        else:
            scanners[-1].add_beacon(Beacon(*l))
    return scanners

def reduce_scanners(scanners):
    base = scanners.pop()
    scanner_positions = []
    while scanners:
        alignment = [how_similar(base.fingerprint(), s.fingerprint()) for s in scanners]
        scanner_index = alignment.index(max(alignment))
        to_align = scanners[scanner_index]
        rotation_matrix, translation = calculate_rotation_and_translation(base, to_align)
        aligned = to_align.transform(rotation_matrix, translation)
        base.add_scanner(aligned)
        del scanners[scanner_index]
        scanner_positions.append(translation)
    return base, scanner_positions

In [34]:
scanners = make_scanners(parse_text(SAMPLE_DIFFERENT_SCANNERS))
result, scanner_positions = reduce_scanners(scanners)
len(result.beacons), scanner_positions

(79,
 [[1104, 88, -113], [1061, 20, 1133], [1081, -72, -1247], [-168, 1125, -72]])

In [35]:
scanners = make_scanners(parse_text(read_input()))
result, scanner_positions = reduce_scanners(scanners)
len(result.beacons), scanner_positions

(496,
 [[-50, -1279, -1],
  [-32, -1155, 1129],
  [-58, -2423, -27],
  [116, -1276, -1304],
  [1309, 28, -14],
  [1263, -1257, -5],
  [1291, 78, -1197],
  [1210, 34, -2532],
  [2461, -1240, -27],
  [2441, -44, -129],
  [2419, -61, -1224],
  [3603, -19, -1237],
  [3639, 1108, -1312],
  [3571, 1128, 48],
  [2343, 1094, -20],
  [2421, -14, 1253],
  [3721, -12, 1081],
  [2372, 2321, -68],
  [3638, 2388, -1277],
  [4835, 46, 1103],
  [4879, -17, -1227],
  [4797, 1232, -1284],
  [3678, 1201, -2499],
  [3699, 47, -2371],
  [4788, -1144, -1313],
  [3597, -1254, -2434],
  [2505, 66, -3579],
  [3703, -29, -3594],
  [3612, 0, -4788],
  [-1135, -1250, 41],
  [-1171, 40, -23],
  [-2303, -1198, 1],
  [3594, -1191, -4879],
  [3589, -77, -6071],
  [3717, -59, -7318],
  [2478, -2467, -114],
  [3586, -2414, -117],
  [2366, -2496, -1215],
  [2518, -2463, -2354]])

In [36]:
def distance_taxi(point1, point2):
    x = point1[0] - point2[0]
    y = point1[1] - point2[1]
    z = point1[2] - point2[2]

    return abs(x) + abs(y) + abs(z)

In [37]:
distances = []
for s1 in scanner_positions:
    for s2 in scanner_positions:
        distances.append(distance_taxi(s1, s2))
max(distances)

14478